In [1]:
#importing packages

import matplotlib.pyplot as plt
import numpy as np
import os
import librosa
import keras
from keras import layers
import keras.backend as K
import random
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers

Using TensorFlow backend.


In [2]:
MUSIC = r'C:\Users\user\Desktop\music\genres_original'
music_dataset = []
genre_target = []

for i in os.listdir(MUSIC):
    for j in os.listdir(MUSIC + "\\" + i):
        if i == 'metal':
            y,_ = librosa.load(MUSIC + "\\" + i + "\\" + j)
            length = len(y)
            for j in range(6):
                if (j+1)*5*22050<=length:
                    music_dataset.append(y[j*5*22050:(j+1)*5*22050])
                    genre_target.append(0)
        elif i == 'blues':
            y,_ = librosa.load(MUSIC + "\\" + i + "\\" + j)
            length = len(y)
            for j in range(6):
                if (j+1)*5*22050<=length:
                    music_dataset.append(y[j*5*22050:(j+1)*5*22050])
                    genre_target.append(1)
        elif i == 'classical':
            y,_ = librosa.load(MUSIC + "\\" + i + "\\" + j)
            length = len(y)
            for j in range(6):
                if (j+1)*5*22050<=length:
                    music_dataset.append(y[j*5*22050:(j+1)*5*22050])
                    genre_target.append(2)    
        elif i == 'hiphop':
            y,_ = librosa.load(MUSIC + "\\" + i + "\\" + j)
            length = len(y)
            for j in range(6):
                if (j+1)*5*22050<=length:
                    music_dataset.append(y[j*5*22050:(j+1)*5*22050])
                    genre_target.append(3)
        elif i == 'pop':
            y,_ = librosa.load(MUSIC + "\\" + i + "\\" + j)
            length = len(y)
            for j in range(6):
                if (j+1)*5*22050<=length:
                    music_dataset.append(y[j*5*22050:(j+1)*5*22050])
                    genre_target.append(4)                   

In [3]:
total_no = len(music_dataset)
print(total_no)

2996


In [4]:
zcr = np.zeros((total_no,216))
centroid = np.zeros((total_no,216))
rolloff = np.zeros((total_no,216))
bandwidth=np.zeros((total_no,216))

mfcc = np.zeros((total_no,20,216))

for i in range(total_no):
    zcr[i] = librosa.feature.zero_crossing_rate(music_dataset[i])[0]
    centroid[i] = librosa.feature.spectral_centroid(y=music_dataset[i])[0]
    rolloff[i] = librosa.feature.spectral_rolloff(y=music_dataset[i])[0]
    bandwidth[i] = librosa.feature.spectral_bandwidth(y=music_dataset[i])[0]
    mfcc[i] = librosa.power_to_db(librosa.feature.mfcc(y=music_dataset[i], sr=22050), ref=np.max)

In [5]:
#changing the numeric features into an image of size 4,216
numeric = np.zeros((total_no,4,216,1))
numeric[:,0,:,0] = zcr
numeric[:,1,:,0] = centroid/11025
numeric[:,2,:,0] = rolloff/11025
numeric[:,3,:,0] = bandwidth/11025

In [6]:
#Shuffling the dataset
X_numeric = np.zeros((total_no,4,216,1))
X_mfcc = np.zeros((total_no,20,216,1))
Y = np.zeros(len(genre_target))

shuffle_index = [i for i in range(total_no)]
random.shuffle(shuffle_index)
for i in range(total_no):
    X_numeric[i] = numeric[shuffle_index[i]]
    X_mfcc[i,:,:,0] = mfcc[shuffle_index[i]]/80 + 1
    Y[i] = genre_target[shuffle_index[i]]

In [7]:
#splitting the dataset to train and test set
X_train_numeric = X_numeric[:2700]
X_train_mfcc = X_mfcc[:2700]
Y_train = Y[:2700]

X_val_numeric = X_numeric[2700:2850]
X_val_mfcc = X_mfcc[2700:2850]
Y_val = Y[2700:2850]

X_test_numeric = X_numeric[2850:]
X_test_mfcc = X_mfcc[2850:]
Y_test = Y[2850:]

In [13]:
numeric = keras.Input(shape=(4,216,1))
mfcc = keras.Input(shape=(20,216,1))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(numeric)
x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
x = keras.Model(inputs=numeric, outputs=x)

y = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(mfcc)
y = layers.MaxPooling2D((2, 4), padding='same')(y)
y = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(y)
y = layers.MaxPooling2D((2, 4), padding='same')(y)
y = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(y)
y = layers.MaxPooling2D((2, 4), padding='same')(y)
y = layers.Flatten()(y)
y = layers.Dense(256, activation="relu")(y)
y = keras.Model(inputs=mfcc, outputs=y)

combined = layers.concatenate([x.output, y.output])
z = layers.Dense(64, activation="relu")(combined)
z = layers.Dense(5, activation="softmax")(z)

model = keras.Model(inputs=[x.input, y.input], outputs=z)

In [14]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
# Fitting our neural network
mc_loss = ModelCheckpoint(r"C:\Users\user\Desktop\best_genre_model_loss", monitor='val_loss', mode='min', verbose=0, save_best_only=True)
mc_acc = ModelCheckpoint(r"C:\Users\user\Desktop\best_genre_model_accuracy", monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)
model.fit([X_train_numeric,X_train_mfcc]
          ,Y_train,epochs = 20
          ,batch_size=512
          ,validation_data=([X_val_numeric,X_val_mfcc],Y_val)
          ,callbacks = [mc_loss,mc_acc])

Train on 2700 samples, validate on 150 samples
Epoch 1/20
2700/2700 [==============================] - 6s 2ms/step - loss: 0.1827 - accuracy: 0.9311 - val_loss: 0.4389 - val_accuracy: 0.8133
Epoch 2/20
2700/2700 [==============================] - 6s 2ms/step - loss: 0.1580 - accuracy: 0.9430 - val_loss: 0.4535 - val_accuracy: 0.8133
Epoch 3/20
2700/2700 [==============================] - 6s 2ms/step - loss: 0.1761 - accuracy: 0.9396 - val_loss: 0.4672 - val_accuracy: 0.8200
Epoch 4/20
2700/2700 [==============================] - 7s 3ms/step - loss: 0.1589 - accuracy: 0.9433 - val_loss: 0.5279 - val_accuracy: 0.7800
Epoch 5/20
2700/2700 [==============================] - 6s 2ms/step - loss: 0.1740 - accuracy: 0.9289 - val_loss: 0.5183 - val_accuracy: 0.7867
Epoch 6/20
2700/2700 [==============================] - 6s 2ms/step - loss: 0.1504 - accuracy: 0.9459 - val_loss: 0.4016 - val_accuracy: 0.8267
Epoch 7/20
2700/2700 [==============================] - 7s 3ms/step - loss: 0.1214 - accu

In [16]:
test_loss, test_acc = model.evaluate([X_test_numeric,X_test_mfcc],Y_test)
print("Final model accuracy - " + str(test_acc))
model1 = keras.models.load_model(r"C:\Users\user\Desktop\best_genre_model_accuracy")
test_loss, test_acc = model1.evaluate([X_test_numeric,X_test_mfcc],Y_test)
print("Best accuracy model accuracy - " + str(test_acc))
model1 = keras.models.load_model(r"C:\Users\user\Desktop\best_genre_model_loss")
test_loss, test_acc = model1.evaluate([X_test_numeric,X_test_mfcc],Y_test)
print("Best accuracy model loss - " + str(test_acc))

146/146 [==============================] - 0s 1ms/step
Final model accuracy - 0.7945205569267273
146/146 [==============================] - 0s 1ms/step
Best accuracy model accuracy - 0.8287671208381653
146/146 [==============================] - 0s 1ms/step
Best accuracy model loss - 0.8287671208381653
